In [19]:
# http://dlib.net/face_landmark_detection.py.html
import os
import glob
import dlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import * 
import numpy as np
from face_feature_v2 import Feature

In [20]:
# 5 face landmarks model is used to crop and align faces
# 68 face landamarks model is used to detect precise landmark points on faces
predictor_fl5_path = "face_detection_model/shape_predictor_5_face_landmarks.dat"
predictor_fl68_path = "face_detection_model/shape_predictor_68_face_landmarks.dat"

stroke_folder_path = "Strokefaces/stroke"
non_stroke_folder_path = "Strokefaces/non_stroke"

# Load all the models we need: a detector to find the faces, a shape predictor
# to find face landmarks so we can precisely localize the face
fl5_detector = dlib.get_frontal_face_detector()
fl5_sp = dlib.shape_predictor(predictor_fl5_path)

fl68_detector = dlib.get_frontal_face_detector()
fl68_sp = dlib.shape_predictor(predictor_fl68_path)

In [21]:
non_stroke_labeled_paths = [(f, False) for f in glob.glob(os.path.join(non_stroke_folder_path, "*.jpg"))]
stroke_labeled_paths = [(f, True) for f in glob.glob(os.path.join(stroke_folder_path, "*.jpg"))]

labeled_paths = stroke_labeled_paths + non_stroke_labeled_paths

non_stroke_features = []
stroke_features = []

In [22]:
for f, is_stroke in labeled_paths:
    # Load the image using Dlib
    img = dlib.load_rgb_image(f)
    
    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    fl5_dets = fl5_detector(img, 1)

    num_faces = len(fl5_dets)
    if num_faces == 0:
        print("Sorry, there were no faces found in '{}'".format(f))
        continue

    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    for detection in fl5_dets:
        faces.append(fl5_sp(img, detection))

    # Get aligned and cropped face
    aligned_img = dlib.get_face_chip(img, faces[0])

    fl68_dets = fl68_detector(aligned_img, 2)

    for k, d in enumerate(fl68_dets):
        # Get the landmarks/parts for the face in box d.
        shape = fl68_sp(aligned_img, d)
        ft = Feature(shape.parts())
        if is_stroke == 1:
            stroke_features.append(ft.feature.copy())
        else:
            non_stroke_features.append(ft.feature.copy())

stroke_features = np.array(stroke_features)
non_stroke_features = np.array(non_stroke_features)
print("Finish feature extration")

Finish feature extration


In [23]:
print(len(stroke_features))
print(len(non_stroke_features))

62
89


In [24]:
# 1 = stroke 0 = non_stroke
stroke_labels = np.array([1] * len(stroke_features))
# stroke_dataset = tf.data.Dataset.from_tensor_slices((stroke_features, stroke_labels))

non_stroke_labels = np.array([0] * len(non_stroke_features))
# non_stroke_dataset = tf.data.Dataset.from_tensor_slices((non_stroke_features, non_stroke_labels))

all_features = np.concatenate((stroke_features, non_stroke_features))
all_labels = np.concatenate((stroke_labels, non_stroke_labels))

import random
zipped_feature_labels = list(zip(all_features, all_labels))
random.shuffle(zipped_feature_labels)
all_features, all_labels = zip(*(zipped_feature_labels))

In [65]:
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.metrics import accuracy_score, confusion_matrix
model = svm.SVC()
nFold = 7

kf = KFold(n_splits=nFold, shuffle=False, random_state=None)

ACC_SUM = 0
for i, (train_indices, test_indices) in enumerate(kf.split(all_features)):
    train_features = [all_features[i] for i in train_indices]
    test_features = [all_features[i] for i in test_indices]
    train_labels = [all_labels[i] for i in train_indices]
    test_labels = [all_labels[i] for i in test_indices]

    model.fit(train_features, train_labels)
    
    predicted = model.predict(test_features)

    ac = accuracy_score(predicted, test_labels)
    cm = confusion_matrix(test_labels, predicted)

    # 0 non-stroke 1 stroke
    TN = cm[0][0]
    FN = cm[1][0]
    TP = cm[1][1]
    FP = cm[0][1]
    
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    ACC_SUM += ACC
    print(f"{i} Fold : TN:{TN} FN:{FN} TP:{TP} FP:{FP} Accuracy:{ACC}")
print(f"Average accuracy: {ACC_SUM / nFold}")

0 Fold : TN:14 FN:0 TP:8 FP:0 Accuracy:1.0
1 Fold : TN:13 FN:1 TP:7 FP:1 Accuracy:0.9090909090909091
2 Fold : TN:10 FN:2 TP:10 FP:0 Accuracy:0.9090909090909091
3 Fold : TN:13 FN:0 TP:9 FP:0 Accuracy:1.0
4 Fold : TN:11 FN:2 TP:7 FP:1 Accuracy:0.8571428571428571
5 Fold : TN:14 FN:2 TP:5 FP:0 Accuracy:0.9047619047619048
6 Fold : TN:12 FN:1 TP:8 FP:0 Accuracy:0.9523809523809523
Average: 0.9332096474953618


In [26]:
# all_dataset = stroke_dataset.concatenate(non_stroke_dataset)

# Shuffle the combined dataset
# all_dataset = all_dataset.shuffle(buffer_size=len(stroke_dataset) + len(non_stroke_dataset))

In [27]:
# data_len = len(all_dataset)
# train_len = int(data_len * 0.8)
# test_len = int((data_len - train_len) / 2)
# validate_len = data_len - train_len - test_len

# train_dataset = all_dataset.take(train_len).batch(32)
# test_dataset = (all_dataset.skip(train_len)).take(test_len).batch(32)
# validate_dataset = all_dataset.skip(train_len + test_len).batch(32)

# print(len(train_dataset))
# print(len(test_dataset))
# print(len(validate_dataset))


In [96]:
# num_classes = 2 # non-stroke / stroke
# batch_size = 128
# hidden_units = 256
# dropout = 0.45

# model = tf.keras.Sequential()
# model.add(Dense(hidden_units, input_dim=5))
# model.add(Activation('relu'))
# model.add(Dropout(dropout))
# model.add(Dense(hidden_units))
# model.add(Activation('relu'))
# model.add(Dropout(dropout))
# model.add(Dense(num_classes))

# model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

In [97]:
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.0001)

# model.fit(
#     x=np.array(all_features),
#     y=np.array(all_labels),
#     validation_split=0.2,
#     epochs=50
# )

AttributeError: 'SVC' object has no attribute 'optimizer'

In [36]:
# test_loss, test_accuracy = model.evaluate(test_dataset)

AttributeError: 'SVC' object has no attribute 'evaluate'